In [1]:
#Taller mercado laboral GEIH 2025
#Hecho por: Santiago Niño Duque y Santiago Montoya

In [2]:
#Primero vamos a importar las librerias a usar
import pandas as pd 
import numpy as np

#Importando tambien los datos de la GEIH 2025, directamente desde el repositorio del profesor
df_cgsse = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                       ,encoding="latin", sep= ";") 
df_ft = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_ocupado = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                       ,encoding="latin", sep= ";", low_memory=False) 
df_desocupados = pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                       ,encoding="latin", sep= ";") 

1. Calculando la poblacion total de colombia y los otros datos solicitados

In [3]:
# 1. Calculando la poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [4]:
#Poblacion en edad de trabajar 

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum() #PET


print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(edad_trabajar))


La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [5]:
#Calculado ahora la poblacion economicamente activa
#definimos como poblacion activamente economica como 

#Tenemos que usar el DSI dado que en el modulo de no ocupados contiene informacion de personas que no estan buscando trabajo 
#No ocupados para este analisis son los que estan buscando trabajo activamente DSI=1 
pea = df_ocupado["FEX_C18"].sum() + df_desocupados[df_desocupados['DSI']==1]["FEX_C18"].sum()

#Tambien podriamos usar la variable FT=1 que es la fuerza de trabajo que cumple con las condiciones 
#Tiene trabajo o esta buscando trabajo

#definimos la poblacion economicamente inactiva como
#Usamos la columna FFT que son las personas fuera de la fuerza de trabajo 
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()

print(f'La poblacion economicamente activa es:', int(pea))
print(f'La poblacion economicamente inactiva es:', int(pei))

La poblacion economicamente activa es: 25974846
La poblacion economicamente inactiva es: 14678569


In [6]:
# Ahora vamos a calcular la tasa de ocupacion (TO)
TO = (df_ocupado['FEX_C18'].sum() / edad_trabajar)

print(f'La tasa de ocupacion: {TO:.2%}')

#Calculando el desempleo (TD) tenemos que

TD = (df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum() / pea)
print(f'La tasa de desempleo es {TD:.2%}')

La tasa de ocupacion: 58.43%
La tasa de desempleo es 8.55%


2. Ahora vamos a calcular la tasa de desempleo para hombres y mujeres por departamento

In [ ]:
#Vamos a hacer un merge para que en el dataframe de desocupados quede genero
#Muy importante no olvidar el DSI==1
df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1], df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')
#Ahora agrupamos el departamento y genero con el factor expansion
desocupados_sexo = (df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'desocupados'}))
#No olvidar el reset index para evitar errores y que sea mas facil de trabajar 


#Ahora ponemos el genero en los ocupados
df_ocupados_gen = pd.merge(df_ocupado, df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')
#Aqui agrupamos departamento y genero con el factor de expension
ocupados_sexo = (df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18'].sum().reset_index().rename(columns={'FEX_C18':'ocupados'}))

#Ocupados y Desocupados para poder sacar la PEA
pea_genero = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0) #Fillna para rellenar los espacios vacios
pea_genero['PEA'] = pea_genero['ocupados'] + pea_genero['desocupados']

#Tasa de desempleo
pea_genero['TD'] = pea_genero['desocupados'] / pea_genero['PEA'] * 100

#Ahora poniendo etiquetas para cambiar los valores en la columna
# 1= Hombres 2=Mujeres
pea_genero['Sexo'] = pea_genero['P3271'].map({1:'Hombres', 2:'Mujeres'})

# Mostramos la tabla final
print(pea_genero[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [13]:
#Ahora poniendolo en orden de mayor a menor
td_max = pea_genero.loc[pea_genero.groupby("Sexo")["TD"].idxmax()]
td_min = pea_genero.loc[pea_genero.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo por sexo:")
print(td_max[['DPTO','Sexo','TD']])

print("Departamentos con MENOR tasa de desempleo por sexo:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo por sexo:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892
Departamentos con MENOR tasa de desempleo por sexo:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


Podemos ver que el departamento con mayor desempleo lo que va en el 2025 es Arauca y con menor desempleo es Amazonas

3. Tomando en consideración únicamente los siguientes grupos de edad: 

18 a 24,
24 a 30,
30 a 40,
40 a 50,
60 en adelante 

Para cada uno de los departamentos, cual es el grupo de edad con mayor y menor tasa de desempleo?


In [ ]:
#Vamos a definir una funcion que nos permita dividir las edades por grupos determinados 

def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return "Otros"  # Para menores de 18 y personas entre 50-59 que no entran

In [15]:
#Vamos a crear una columna llamda grupo de edad para meter la nuevea funcion y aplicarla 
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

#metemos nuestra nueva columnda del grupo edad
df_desocupados_gen = pd.merge(df_desocupados[df_desocupados["DSI"]==1],df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],how="left")
desocupados_edad = (df_desocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupados"}))

#Hacemos nueva columna con edades determinadas
df_ocupados_gen = pd.merge(df_ocupado,df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],how="left")
ocupados_edad = (df_ocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupados"}))

#Ahora calculando la pea y la td
pea_edad = pd.merge(ocupados_edad, desocupados_edad, on=["DPTO","grupo_edad"], how="outer").fillna(0)
pea_edad["PEA"] = pea_edad["ocupados"] + pea_edad["desocupados"]
pea_edad["TD"] = pea_edad["desocupados"] / pea_edad["PEA"] * 100

In [18]:
#Ahora filtamos solo para los grupos de nuestro interes 
pea_edad = pea_edad[pea_edad["grupo_edad"].isin(["18-24","24-30","30-40","40-50","60+"])] 

#Ahora filtramos por departamento para mayores y menores con .idxmax e .idxmin
mayores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmax()]
menores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmin()]

print("Grupo con MAYOR tasa de desempleo por departamento:")
print(mayores[["DPTO","grupo_edad","TD"]])

print("Grupo con MENOR tasa de desempleo por departamento:")
print(menores[["DPTO","grupo_edad","TD"]])

Grupo con MAYOR tasa de desempleo por departamento:
     DPTO grupo_edad         TD
0       5      18-24  17.685962
6       8      18-24  22.406918
12     11      18-24  17.797990
18     13      18-24  29.157386
25     15      24-30  14.491221
30     17      18-24   9.972584
36     18      18-24  21.686963
43     19      24-30   7.775308
48     20      18-24  19.042303
54     23      18-24  31.660884
60     25      18-24  15.618810
66     27      18-24  21.102675
72     41      18-24  14.602874
79     44      24-30  24.685637
84     47      18-24  20.675536
90     50      18-24  13.483416
96     52      18-24  16.529172
102    54      18-24  28.074930
109    63      24-30  14.237924
114    66      18-24  15.173556
120    68      18-24  23.222696
127    70      24-30  22.884184
132    73      18-24  19.348020
138    76      18-24  17.821703
144    81      18-24  61.095429
150    85      18-24  27.966955
156    86      18-24  44.878068
163    88      24-30  23.449477
171    91      40-50

En la mayoría de los departamentos, el grupo con mayor tasa de desempleo corresponde a los jóvenes de 18 a 24 años, lo que refleja las dificultades de inserción laboral de esta población.

4. ¿Cuáles son los 3 departamentos con menor tasa de desempleo de mujeres entre los 18 y los 25 años ? 

In [23]:
#Vamos a filtrar para las mujeres entre 18 y 25 años
# Paso 1: crear grupos de edad
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

#usamos el df_ocupados_gen, ya creado anteriormente pero poniendo el p3271

df_ocupados_gen2 = pd.merge(df_ocupado,df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],how="left")
ocupados_edad = (df_ocupados_gen2.groupby(["DPTO","P3271"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupados"}))

#Lo mismo con los desocupados

df_desocupados_gen2 = pd.merge(df_desocupados,df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],how="left")
ocupados_edad = (df_ocupados_gen2.groupby(["DPTO","P3271"])["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupados"}))

#Aqui ya aplicamos el filtro
df_mujeres_oc = df_ocupados_gen2[(df_ocupados_gen2["P3271"]==2) & (df_ocupados_gen2["grupo_edad"]=="18-24")]
df_mujeres_des = df_desocupados_gen2[(df_desocupados_gen2["P3271"]==2) & (df_desocupados_gen2["grupo_edad"]=="18-24")]

#Agrupamos por departamento
ocupadas = df_mujeres_oc.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupadas"})
desocupadas = df_mujeres_des.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupadas"})

#Ahora con esto calculamos la pea y el td
pea = pd.merge(ocupadas, desocupadas, on="DPTO", how="outer").fillna(0)
pea["PEA"] = pea["ocupadas"] + pea["desocupadas"]
pea["TD"] = pea["desocupadas"] / pea["PEA"] * 100

#Y mostramos los tres de menor valor
td_mujeres_min = pea.sort_values("TD").head(3)
print(td_mujeres_min)

    DPTO      ocupadas   desocupadas            PEA         TD
30    95   1218.212804    649.874589    1868.087394  34.788233
10    25  98335.776355  57867.376219  156203.152574  37.046228
32    99    510.425025    365.704182     876.129207  41.740896


Las mujeres con menor tasa de desempleo son las de Vaupes, Amazonas y Quindio 

5. Calcule el promedio de ingresos laborales  de las personas residentes en los departamentos de Risaralda, Caldas , Quindío, Antioquia y Valle del Cauca.

In [25]:
#definimos los departamentos a usar 
deptos_interes = [66, 17, 63, 5, 76]  # 66=Risaralda, 17=Caldas, 63=Quindío, 5=Antioquia, 76=Valle del Cauca

df_ingresos = df_ocupado[df_ocupado["DPTO"].isin(deptos_interes)]

# Aqui calculamos el promedio ponderado de ingresos con la variable INGLABO
promedios_ingresos = (
    df_ingresos.groupby("DPTO")[["INGLABO","FEX_C18"]]
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

#Vamos a cambiar los numeros cientificos por numero entero
promedios_ingresos["Ingreso_promedio"] = promedios_ingresos["Ingreso_promedio"].map("{:,.0f}".format)

# Reemplazar códigos de departamentos por nombres
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios_ingresos["Departamento"] = promedios_ingresos["DPTO"].map(mapa_dptos)

# Reordenar columnas
promedios_ingresos = promedios_ingresos[["Departamento","Ingreso_promedio"]]

print(promedios_ingresos)

      Departamento Ingreso_promedio
0        Antioquia        2,090,819
1           Caldas        1,958,683
2          Quindío        1,637,336
3        Risaralda        1,621,205
4  Valle del Cauca        1,698,392


Lo resultados dan que los ingresos promedio para Antioquia son 2.090.819 pesos, para Caldas son 1.958.683 pesos, para quindio 1.637.336, para Risaralda 1.621.205 y para el valle del cauca 1.698.392, dandonos asi que el mayor ingreso promedio es para Antioquia y el menor para Risaralda

6. Para los mismos departamentos del punto anterior, calcule el promedio de ingresos laborales de hombres y mujeres en los siguientes rangos de edad : 
18 a 24,
24 a 30,
30 a 40,
40 a 50 ,
60 en adelante 


In [26]:
#Vamos a aplicar un dataframe filtrado solo con los deptos de interes
df_filtrado = df_ocupados_gen2[df_ocupados_gen2["DPTO"].isin(deptos_interes)]

In [34]:
# Aqui calculamos el promedio ponderado de ingresos con la variable INGLABO en los departamentos de interes
promedios_ingresos_dtos = (
    df_filtrado.groupby(["DPTO","P3271","grupo_edad"])[["INGLABO","FEX_C18"]]
      .apply(lambda g: (g["INGLABO"] * g["FEX_C18"]).sum() / g["FEX_C18"].sum())
      .reset_index(name="Ingreso_promedio")
)


#Vamos a cambiar los numeros cientificos por numero entero
promedios_ingresos_dtos["Ingreso_promedio"] = promedios_ingresos_dtos["Ingreso_promedio"].map("{:,.0f}".format)

# Reemplazar códigos de departamentos por nombres
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios_ingresos_dtos["Departamento"] = promedios_ingresos_dtos["DPTO"].map(mapa_dptos)

# Reordenar columnas
promedios_ingresos_dtos = promedios_ingresos_dtos[["Departamento","grupo_edad","Ingreso_promedio"]]

print(promedios_ingresos_dtos)

       Departamento grupo_edad Ingreso_promedio
0         Antioquia      18-24        1,364,005
1         Antioquia      24-30        1,828,144
2         Antioquia      30-40        2,159,206
3         Antioquia      40-50        2,771,747
4         Antioquia        60+        1,738,620
5         Antioquia      Otros        2,554,671
6         Antioquia      18-24        1,418,639
7         Antioquia      24-30        1,837,916
8         Antioquia      30-40        2,291,801
9         Antioquia      40-50        2,111,094
10        Antioquia        60+        1,510,449
11        Antioquia      Otros        1,790,046
12           Caldas      18-24        2,216,063
13           Caldas      24-30        1,754,677
14           Caldas      30-40        1,762,301
15           Caldas      40-50        2,470,039
16           Caldas        60+        2,309,864
17           Caldas      Otros        1,669,417
18           Caldas      18-24        1,133,456
19           Caldas      24-30        1,